In [ ]:
#Notbook virtuelle Pythonumgebung erstellen und notebook kernel erstellen

In [ ]:
!python -m venv venv
!venv\Scripts\python -m pip install jupyter
!venv\Scripts\python -m pip install ipykernel
!venv\Scripts\python -m ipykernel install --user --name=venv --display-name "Python (venv)"

In [ ]:
# Installation der Bibliotheken

In [ ]:
!pip install opencv-python
!pip install opencv-python-headless
!pip install numpy
!pip installpillow
!pip install tensorflow

In [ ]:
# Modell ausführen

In [4]:
import os
import numpy as np
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras import models
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import moviepy.editor as mpy

# Modellpfad und Konfigurationsparameter
model_dir = "llm"
model_path = os.path.join(model_dir, "transformation_model.keras")
output_folder = "generated_frames"
os.makedirs(output_folder, exist_ok=True)

# Lade das Modell
if os.path.exists(model_path):
    model = models.load_model(model_path)
else:
    model = None
    print("Kein Modell gefunden. Bitte lade das trainierte Modell.")

image_size = (256, 256)
num_frames = 30 * 24  # 30 Sekunden bei 24 FPS

# Funktion zur Generierung der Sequenz mit verschiedenen Transformationen
def generate_sequence(input_image_path, model, transformation_type, intensity, num_frames=100):
    img = Image.open(input_image_path).resize(image_size).convert("RGB")
    img_array = np.array(img) / 255.0
    frames = []
    current_frame = img_array

    for i in range(num_frames):
        next_frame = model.predict(np.expand_dims(current_frame, axis=0))[0]
        
        if transformation_type == "Farbverschiebung":
            hsv_image = cv2.cvtColor((next_frame * 255).astype(np.uint8), cv2.COLOR_RGB2HSV)
            hsv_image[:, :, 0] = (hsv_image[:, :, 0] + intensity * (i % 180)) % 180
            next_frame = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB) / 255.0
        
        elif transformation_type == "Helligkeitsvariation":
            next_frame = (next_frame * (1 + intensity * (i / num_frames))) % 1.0
        
        elif transformation_type == "Kontrastverstärkung":
            next_frame = np.clip((next_frame - 0.5) * (1 + intensity) + 0.5, 0, 1)
        
        frames.append((next_frame * 255).astype(np.uint8))
        current_frame = next_frame

    return frames

# GUI-Klasse für die Anwendung
class ImageTransformerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Bildtransformation und Videoerstellung")
        self.root.geometry("700x500")

        self.input_image_path = None
        self.transformation_type = tk.StringVar(value="Standard")
        self.intensity = tk.DoubleVar(value=0.5)  # Intensity-Einstellung für Transformationen

        self.create_widgets()

    def create_widgets(self):
        tk.Label(self.root, text="Wähle ein Bild zur Transformation:").pack(pady=5)
        tk.Button(self.root, text="Bild auswählen", command=self.select_image).pack(pady=5)

        tk.Label(self.root, text="Transformationstyp auswählen:").pack(pady=5)
        tk.OptionMenu(self.root, self.transformation_type, "Standard", "Farbverschiebung", "Helligkeitsvariation", "Kontrastverstärkung").pack(pady=5)

        tk.Label(self.root, text="Intensität der Transformation:").pack(pady=5)
        tk.Scale(self.root, variable=self.intensity, from_=0.1, to=1.5, resolution=0.1, orient="horizontal").pack(pady=5)

        tk.Button(self.root, text="Sequenz generieren und als Video speichern", command=self.generate_video).pack(pady=20)

        self.canvas = tk.Canvas(self.root, width=256, height=256)
        self.canvas.pack(pady=20)

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
        if file_path:
            self.input_image_path = file_path
            img = Image.open(self.input_image_path).resize((256, 256))
            self.display_image(img)

    def display_image(self, img):
        img_tk = ImageTk.PhotoImage(img)
        self.canvas.create_image(128, 128, image=img_tk)
        self.canvas.image = img_tk

    def generate_video(self):
        if not self.input_image_path:
            messagebox.showwarning("Warnung", "Bitte wähle zuerst ein Eingabebild.")
            return

        if model is None:
            messagebox.showwarning("Warnung", "Kein Modell geladen. Trainiere zuerst ein Modell.")
            return

        transformation_type = self.transformation_type.get()
        intensity = self.intensity.get()
        frames = generate_sequence(self.input_image_path, model, transformation_type, intensity, num_frames=num_frames)

        for i, frame in enumerate(frames):
            frame_image = Image.fromarray(frame)
            frame_image.save(os.path.join(output_folder, f"frame_{i:04d}.png"))

        video_path = "output_video.mp4"
        self.create_video_from_frames(video_path)
        messagebox.showinfo("Fertig", f"Video wurde unter {video_path} gespeichert.")

    def create_video_from_frames(self, video_path):
        frame_files = [os.path.join(output_folder, f"frame_{i:04d}.png") for i in range(num_frames)]
        clips = [mpy.ImageClip(m).set_duration(1/24) for m in frame_files]
        video = mpy.concatenate_videoclips(clips, method="compose")
        video.write_videofile(video_path, fps=24)

# Start der Anwendung
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageTransformerApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━

Moviepy - Done !
Moviepy - video ready output_video.mp4


In [2]:
# Modell Training


In [6]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
import tensorflow as tf

# Ordner mit den Trainingsbildern
data_dirs = {
    "growth_and_spread": "output_frames/growth_and_spread",
    "light_variation": "output_frames/light_variation",
    "organic_movement": "output_frames/organic_movement",
    "metamorphosis": "output_frames/metamorphosis"
}

# Modell-Speicherort
model_dir = "llm"
model_path = os.path.join(model_dir, "transformation_model.keras")
os.makedirs(model_dir, exist_ok=True)

# Bildgröße und Parameter
image_size = (256, 256)
batch_size = 16
epochs = 20

# Daten-Generator-Klasse
class DataGenerator(Sequence):
    def __init__(self, data_dirs, image_size, batch_size):
        self.data_dirs = data_dirs
        self.image_size = image_size
        self.batch_size = batch_size
        self.file_paths, self.labels = self.get_file_paths_and_labels()
        self.on_epoch_end()

    def get_file_paths_and_labels(self):
        file_paths = []
        labels = []
        for label, dir_name in enumerate(self.data_dirs.values()):
            files = sorted([os.path.join(dir_name, f) for f in os.listdir(dir_name) if f.endswith('.jpg') or f.endswith('.png')])
            file_paths.extend(files[:-1])
            labels.extend(files[1:])
        return file_paths, labels

    def __len__(self):
        return int(np.floor(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_file_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(batch_file_paths, batch_labels)
        return X, y

    def __data_generation(self, batch_file_paths, batch_labels):
        X = np.empty((self.batch_size, *self.image_size, 3))
        y = np.empty((self.batch_size, *self.image_size, 3))
        
        for i, (file_path, label_path) in enumerate(zip(batch_file_paths, batch_labels)):
            img = Image.open(file_path).resize(self.image_size).convert("RGB")
            img_array = np.array(img) / 255.0
            X[i] = img_array
            
            label_img = Image.open(label_path).resize(self.image_size).convert("RGB")
            y[i] = np.array(label_img) / 255.0
        
        return X, y

# Erstelle den Generator
train_generator = DataGenerator(data_dirs, image_size, batch_size)

# Erstelle das Modell oder lade ein vorhandenes Modell
if os.path.exists(model_path):
    print("Lade vorhandenes Modell...")
    model = tf.keras.models.load_model(model_path)
else:
    print("Erstelle ein neues Modell...")
    model = models.Sequential([
        layers.Input(shape=(*image_size, 3)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # RGB-Ausgabe
    ])
    model.compile(optimizer='adam', loss='mse')

# Modelltraining mit dem Generator
print("Beginne mit dem Training...")
model.fit(train_generator, epochs=epochs)

# Speichere das trainierte Modell
model.save(model_path)
print(f"Modell gespeichert unter {model_path}")


Lade vorhandenes Modell...
Beginne mit dem Training...
Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - loss: 0.0082
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0160
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0102
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0083
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0072
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - loss: 0.0076
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0064
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - loss: 0.0057
Epoch 9/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - loss: 0.0063
Epoch 10/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0060
Epoch 11/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0054
Epoch 12/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - loss: 0.0065
Epoch 13/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - loss: 0.0065
Epoch 14/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - loss: 0.0054
Epoch 15/20
63/63 ━━━━━━━━━━━━